In [1]:
import pandas as pd

In [5]:
data = pd.read_csv("../../test_models/cr_trans_1500/data/ml_preds/ensemble_trend_probs_cr_trans_forced_P3_34_1_len1500.csv")

In [11]:
data['bif_prob'] = data.iloc[:,[0,1,2]].sum(axis=1)

In [15]:
data.iloc[50:60,[4,5]]

,0.000000000000000000e+00.4,bif_prob
50,0.183128,0.183128
51,0.140353,0.140353
52,0.137865,0.137865
53,0.154233,0.154233
54,0.266696,0.266696
55,0.172825,0.172825
56,0.221218,0.221218
57,0.223486,0.223486
58,0.277392,0.277392
59,0.570550,0.570550


In [28]:
data = pd.read_csv("../../test_models/cr_trans_1500/data/ml_preds_test/df_ml_null.csv", header=0)

In [29]:
data.head()

,fold_prob,hopf_prob,branch_prob,null_prob,bif_prob,tsid,Time
0,0.181656,0.294452,0.316473,0.207418,NaN,1,0.0
1,0.192455,0.252564,0.298552,0.256430,NaN,1,10.0
2,0.205519,0.204364,0.278671,0.311446,NaN,1,20.0
3,0.215933,0.172944,0.279517,0.331606,NaN,1,30.0
4,0.224729,0.142180,0.273556,0.359534,NaN,1,40.0


In [35]:
data[6] = 0

In [36]:
data.head()

,fold_prob,hopf_prob,branch_prob,null_prob,bif_prob,tsid,Time,6
0,0.181656,0.294452,0.316473,0.207418,0.792582,0,0.0,0
1,0.192455,0.252564,0.298552,0.256430,0.743570,0,10.0,0
2,0.205519,0.204364,0.278671,0.311446,0.688554,0,20.0,0
3,0.215933,0.172944,0.279517,0.331606,0.668394,0,30.0,0
4,0.224729,0.142180,0.273556,0.359534,0.640466,0,40.0,0


In [30]:
data['bif_prob'] = data['fold_prob'] + data['hopf_prob'] + data['branch_prob']

In [31]:
data.head()

,fold_prob,hopf_prob,branch_prob,null_prob,bif_prob,tsid,Time
0,0.181656,0.294452,0.316473,0.207418,0.792582,1,0.0
1,0.192455,0.252564,0.298552,0.256430,0.743570,1,10.0
2,0.205519,0.204364,0.278671,0.311446,0.688554,1,20.0
3,0.215933,0.172944,0.279517,0.331606,0.668394,1,30.0
4,0.224729,0.142180,0.273556,0.359534,0.640466,1,40.0


In [32]:
data.to_csv("../../test_models/cr_trans_1500/data/ml_preds_test/df_ml_null.csv", index=False)

In [1]:

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import warnings
warnings.filterwarnings("ignore")

import gc
import numpy as np
import pandas as pd
import random
import sys
import itertools

import tensorflow as tf
from tensorflow.keras.models import load_model
from datetime import datetime
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

random.seed(datetime.now())

In [2]:
def get_dl_predictions(resids, model_add):
    
    '''
    Generate DL prediction time series on resids
    from DL classifier with type 'model_type' and index kk.
    '''
        
    # Load in specific DL classifier
    model = load_model(model_add)
    y_preds = []
    # Loop through each possible length of padding
    # Start with revelaing the DL algorith only the earliest points
    for pad_count in range(pad_samples-1, -1, -1):
    
        temp_ts = np.zeros((1,ts_len,1))

        ts_gap = ts_len-seq_len
        pad_length = mult_factor*pad_count

        if pad_length + ts_gap > ts_len:
            zero_range = ts_len
        else:
            zero_range = pad_length + ts_gap
        
        if zero_range == ts_len:
            # Set all ML predictions to zero
            y_pred = np.zeros(4).reshape(1,4)
        else:
            for j in range(0, zero_range):
                temp_ts[0,j] = 0
            for j in range(zero_range, ts_len):
                temp_ts[0,j] = resids[0,j-zero_range]

            # normalizing inputs: take averages, since the models were also trained on averaged data. 
            values_avg = 0.0
            count_avg = 0
            for j in range (0,ts_len):
                if temp_ts[0,j] != 0:
                    values_avg = values_avg + abs(temp_ts[0,j])
                    count_avg = count_avg + 1
            if count_avg != 0:
                values_avg = values_avg/count_avg
            for j in range (0,ts_len):
                if temp_ts[0,j] != 0:
                    temp_ts[0,j] = temp_ts[0,j]/values_avg
            
            # Compute DL prediction
            y_pred = model.predict(temp_ts,verbose=0)
            y_preds.append(y_pred)
            
                    
    
    # Delete model and do garbage collection to free up RAM
    tf.keras.backend.clear_session()
    if zero_range != ts_len:
        del model
    gc.collect()
    
    return y_preds

In [6]:
from glob import glob
for index, filepath in enumerate(glob("../../test_models/cr_trans_1500/data/resids/*")):
    print("{} - doing prediction for {}".format(index, filepath))
    filepath_out = f"../test_models/cr_trans_1500/data/my_ml_preds_test/pred_{filepath.split('/')[-1]}"
    ts_len=1500

    # Filepath to export ensemble DL predictions to
    #filepath_out = '../test_models/may_fold_1500/data/ml_preds_test/ensemble_trend_probs_may_fold_null_forced_1_len1500.csv'

    # Type of classifier to use (1500 or 500)
    # ts_len=1500

    '''  
    The following two parameters control how many sample points along the 
    timeseries are used, and the length between them.  For instance, for an input 
    time series equal to or less then length 1500, mult_factor=10 and 
    pad_samples=150 means that it will do the unveiling in steps of 10 datapoints, 
    at 150 evenly spaced points across the entire time series (150 x 10 = 1500).
    Needs to be adjusted according to whether you are using the trained 500 length 
    or 1500 length classifier.
    '''

    # Steps of datapoints in between each DL prediction
    mult_factor = 10

    # Total number of DL predictions to make
    # Use 150 for length 1500 time series. Use 50 for length 500 time series.
    pad_samples = 150



    # Load residual time series data
    df = pd.read_csv(filepath).dropna()
    resids = df['Residuals'].values.reshape(1,-1,1)
    # Length of inupt time series
    seq_len = len(df)

    model_list = [
        "run13"
    ]
    for model_name in model_list:
        model_add = f'../my_best_models/{model_name}/{model_name}.pkl'
        print(f'Compute DL predictions with {model_add}')
    
        pred = get_dl_predictions(resids, model_add)
        break
    break

0 - doing prediction for ../../test_models/cr_trans_1500/data/resids/cr_trans_1500_resids_9.csv
Compute DL predictions with ../my_best_models/run13/run13.pkl


2023-07-15 17:07:34.072899: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:429] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR


UnknownError: Graph execution error:

Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[model/lstm/PartitionedCall]] [Op:__inference_predict_function_59981]